# Clustering Crypto

In [ ]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from pathlib import Path

### Fetching Cryptocurrency Data

In [ ]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [ ]:
# Create a DataFrame 
response_df = requests.get(url).json()

In [ ]:
# Alternatively, use the provided csv file:
path = Path("crypto_data.csv")

# Create a DataFrame
crypto_df = pd.read_csv(path, index_col = 0)

crypto_df.head()

### Data Preprocessing

In [ ]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply']]
print(crypto_df.shape)
crypto_df.tail(30)

In [ ]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df[crypto_df['IsTrading']==True]
crypto_df.shape

In [ ]:
# Keep only cryptocurrencies with a working algorithm
crypto_data_df = crypto_df[crypto_df['Algorithm']==True]
crypto_data_df.shape

In [ ]:
# Remove the "IsTrading" column
crypto_df.drop(['IsTrading'], axis = 1, inplace = True)
crypto_df.head()

In [ ]:
# Remove rows with at least 1 null value
crypto_df = crypto_df[pd.notnull(crypto_df['TotalCoinsMined'])]
crypto_df.head()

In [ ]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df.drop(crypto_df.loc[crypto_df['TotalCoinsMined']==0.000000e+00].index, inplace=True)

In [ ]:
# Drop rows where there are 'N/A' text values
crypto_df.dropna() 

In [ ]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_name = pd.DataFrame(crypto_df["CoinName"], index=crypto_df.index)
coin_name.head()

In [ ]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.drop("CoinName", axis=1, inplace=True)
crypto_df.head()

In [ ]:
# Create dummy variables for text features
X = pd.get_dummies(crypto_df)
X.head()

In [ ]:
# Standardize data
crypto_scaled_data = StandardScaler().fit_transform(X)

### Reducing Dimensions Using PCA

In [ ]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca_df = pca.fit_transform(crypto_scaled_data)

In [ ]:
# Create a DataFrame with the principal components data
df_crypto_pca = pd.DataFrame(
    data=crypto_pca_df, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index)    

df_crypto_pca.head(10)

### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [ ]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_crypto_pca)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_df = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_df)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

Running K-Means with `k=<your best value for k here>`

In [ ]:
# Initialize the K-Means model
model = KMeans(n_clusters=3, random_state=0)

# Fit the model
model.fit(df_crypto_pca)

# Predict clusters
predictions = model.predict(df_crypto_pca)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
df_crypto_pca["class"] = model.labels_
df_crypto_pca.head()

### Visualizing Results

#### 3D-Scatter with Clusters

In [ ]:
#Create a new DataFrame named clustered_df, that includes the following columns "Algorithm", "ProofType", "TotalCoinsMined", "TotalCoinSupply", "PC 1", "PC 2", "PC 3", "CoinName", "Class". You should maintain the index of the crypto_df DataFrames as is shown bellow.
clustered_df = pd.concat([coin_name, crypto_df, df_crypto_pca], axis = 1)
clustered_df.head()

In [ ]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 3",
    y="PC 2",
    z="PC 1",
    hover_name="CoinName",
    hover_data=["Algorithm"],
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos
tradable_cryptos = clustered_df[['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined']]
tradable_cryptos.hvplot.table(width=800)

In [ ]:
# Print the total number of tradable cryptocurrencies
tradable_cryptos = clustered_df.shape[0]
print(tradable_cryptos)

#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scale data to create the scatter plot
scaler = MinMaxScaler()

normalize_columns = ['TotalCoinsMined', 'TotalCoinSupply']
x = crypto_df[normalize_columns].values
crypto_scaled_data = scaler.fit_transform(x)

clustered_df = pd.DataFrame(crypto_scaled_data, columns=normalize_columns, index=crypto_df.index).reset_index()

In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
clustered_df.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    hover_cols=["CoinName"],
)